In [ ]:
import tensorflow_decision_forests as tfdf

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

# Download the dataset
# !wget -q https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins.csv -O /tmp/penguins.csv

# Load a dataset into a Pandas Dataframe.
dataset_df = pd.read_csv("/home/huigang39/e-skin/data/train.csv")

# Display the first 3 examples.
dataset_df.head(3)

In [ ]:
label = "label"

classes = dataset_df[label].unique().tolist()
print(f"Label classes: {classes}")

dataset_df[label] = dataset_df[label].map(classes.index)

In [ ]:
def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(dataset_df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

In [ ]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label)

In [ ]:
# 构建随机森林
model = tfdf.keras.RandomForestModel()

# 训练模型
model.fit(x=train_ds)

# 评估模型
model.compile(metrics=["accuracy"])
evaluation = model.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

In [ ]:
with open("plot.html", "w") as f:
  f.write(tfdf.model_plotter.plot_model(model, tree_idx=0, max_depth=3))

from IPython.display import IFrame
IFrame(src='./plot.html', width=700, height=600)

In [ ]:
import matplotlib.pyplot as plt

logs = model.make_inspector().training_logs()

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Accuracy (out-of-bag)")

plt.subplot(1, 2, 2)
plt.plot([log.num_trees for log in logs], [log.evaluation.loss for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Logloss (out-of-bag)")

plt.show()

In [ ]:
# 预测标签
data = pd.read_csv("/home/huigang39/e-skin/data/test.csv")
data = tfdf.keras.pd_dataframe_to_tf_dataset(data)

import numpy as np

# 假设predictions是模型预测的概率分布
predictions = model.predict(data)

# 使用numpy的argmax函数找到最大概率对应的索引
predicted_label_index = np.argmax(predictions)

print("预测标签:", predicted_label_index)

# 标签到类名的映射
label_to_class = {index: class_name for index, class_name in enumerate(classes)}

# 使用预测的索引获取对应的类名
predicted_class = label_to_class[predicted_label_index]

print("预测分类:", predicted_class)


In [ ]:
# 保存模型
model.save("/home/huigang39/e-skin/model")


In [15]:
# 转换模型为TensorFlow Lite格式
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 保存转换后的模型
with open("converted_model.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp_6n30ir6/assets


INFO:tensorflow:Assets written to: /tmp/tmp_6n30ir6/assets
[INFO 23-06-01 09:14:59.2651 CST kernel.cc:1242] Loading model from path /tmp/tmp_6n30ir6/assets/ with prefix abb72d17f5864b67
[INFO 23-06-01 09:14:59.5810 CST decision_forest.cc:660] Model loaded with 300 root(s), 109944 node(s), and 10 input feature(s).
[INFO 23-06-01 09:14:59.5810 CST abstract_model.cc:1311] Engine "RandomForestGeneric" built
[INFO 23-06-01 09:14:59.5810 CST kernel.cc:1074] Use fast generic engine
2023-06-01 09:14:59.623681: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-06-01 09:14:59.623746: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-06-01 09:14:59.624078: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp_6n30ir6
2023-06-01 09:14:59.629722: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-06-01 09:14:59.629767: I tensorflow/

ConverterError: <unknown>:0: error: loc(fused["SimpleMLCreateModelResource:", "SimpleMLCreateModelResource"]): 'tf.SimpleMLCreateModelResource' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["SimpleMLCreateModelResource:", "SimpleMLCreateModelResource"]): Error code: ERROR_NEEDS_CUSTOM_OPS
/home/huigang39/.local/lib/python3.10/site-packages/tensorflow/python/framework/func_graph.py:707:0: error: 'tf.SimpleMLInferenceOpWithHandle' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall"]): called from
/home/huigang39/.local/lib/python3.10/site-packages/tensorflow/python/framework/func_graph.py:707:0: note: Error code: ERROR_NEEDS_CUSTOM_OPS
<unknown>:0: error: failed while converting: 'main': 
Some ops in the model are custom ops, See instructions to implement custom ops: https://www.tensorflow.org/lite/guide/ops_custom 
Custom ops: SimpleMLCreateModelResource, SimpleMLInferenceOpWithHandle
Details:
